In [6]:
import sys
import pandas as pd
import numpy as np
import os, inspect

sys.path.append('C:\\Users\\alicj\\Documents\\GitHub\\RGCPD')

from RGCPD.class_BivariateMI import BivariateMI
import class_BivariateMI, functions_pp
from stat_models_cont import ScikitModel
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegressionCV
from pipeline.data_preparations import coordinates, data_steps, mean, xarray, month, data_prep
import func_models as fc_utils
from pipeline import pipeline
from RGCPD import core_pp

%load_ext autoreload
%autoreload 2

# main directory is the directory of this notebook
main_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) 

# my main directory (which contains folders with data etc)
my_main_dir = 'C:\\Users\\alicj\\Desktop\\WORK'
data_path = os.path.join(my_main_dir, 'data')
district_data_path = os.path.join(data_path, 'district')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#read_file = pd.read_csv(os.path.join(data_path, 'SA_month_rainfall.txt'))
#read_file.to_csv(os.path.join(data_path, 'SA_month_rainfall.csv'), index = None)

In [8]:
sa = pd.read_csv(os.path.join(data_path, 'data_SA_new.csv'))

In [9]:
sa.columns = sa.columns.str.replace('[X]', '')

In [10]:
sa

,V1,V2,1981.01.01,1981.02.01,1981.03.01,1981.04.01,1981.05.01,1981.06.01,1981.07.01,1981.08.01,...,2020.03.01,2020.04.01,2020.05.01,2020.06.01,2020.07.01,2020.08.01,2020.09.01,2020.10.01,2020.11.01,2020.12.01
0,20.025,-0.025,141.0,155.0,187.0,135.0,170.0,77.0,141.0,166.0,...,184.0,131.0,163.0,110.0,124.0,146.0,152.0,209.0,251.0,253.0
1,20.075,-0.025,141.0,153.0,186.0,136.0,166.0,79.0,145.0,169.0,...,181.0,131.0,158.0,107.0,130.0,150.0,156.0,209.0,251.0,238.0
2,20.125,-0.025,141.0,152.0,182.0,139.0,162.0,79.0,141.0,168.0,...,177.0,132.0,160.0,108.0,126.0,154.0,165.0,210.0,254.0,225.0
3,20.175,-0.025,141.0,150.0,178.0,141.0,157.0,80.0,136.0,168.0,...,173.0,136.0,158.0,112.0,127.0,152.0,162.0,211.0,249.0,210.0
4,20.225,-0.025,141.0,149.0,178.0,144.0,153.0,80.0,136.0,168.0,...,170.0,137.0,154.0,113.0,131.0,152.0,165.0,198.0,246.0,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383995,49.775,-31.975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383996,49.825,-31.975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383997,49.875,-31.975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383998,49.925,-31.975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## define coordinates for 4 pilot districts (Changara, Guija, Marara, Tete)

In [30]:
lon_changara = [33.27024, 33.28388]
lat_changara = [-16.83843, -16.81411]

lon_guija = [32.73913, 33.43538]
lat_guija = [-24.48818, -23.64053]

lon_marara = [32.75777, 33.52642]
lat_marara = [-16.38731, -15.74335]

lon_tete = [33.42356, 33.66757]
lat_tete = [-16.26129, -16.04715]

## get data for each district and target period and save it in nc files

### Changara

#### as for target periods NDJ and DJF there is cross year, additional manipulation is required, hence that data for all target period will be aggregated (to avoid differences in applying the pipeline for different target periods)

In [94]:
def aggregate(data):
    aggr = []
    dates = []
    for i in range(0, len(data), 3):
        aggr.append(sum(data['rainfall'].iloc[i:i+3]))
        dates.append(data.index[i+2])
    return dates, aggr

In [101]:
### OND
changara_ond = data_prep(sa, lon_changara, lat_changara, 'rainfall', [10, 11, 12])
# aggregate
dates, aggr = aggregate(changara_ond)
changara_ond = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(changara_ond).to_netcdf(os.path.join(district_data_path, 'changara_ond.nc'))


### NDJ
changara_ndj = data_prep(sa, lon_changara, lat_changara, 'rainfall', [1, 11, 12]).iloc[1:-2]
# aggregate
dates, aggr = aggregate(changara_ndj)
changara_ndj = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(changara_ndj).to_netcdf(os.path.join(district_data_path, 'changara_ndj1.nc'))


### DJF
changara_djf = data_prep(sa, lon_changara, lat_changara, 'rainfall', [1, 2, 12]).iloc[2:-1]
# aggregate
dates, aggr = aggregate(changara_djf)
changara_djf = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(changara_djf).to_netcdf(os.path.join(district_data_path, 'changara_djf.nc'))


### JFM
changara_jfm = data_prep(sa, lon_changara, lat_changara, 'rainfall', [1, 2, 3])
# aggregate
dates, aggr = aggregate(changara_jfm)
changara_jfm = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(changara_jfm).to_netcdf(os.path.join(district_data_path, 'changara_jfm.nc'))

### Guija

In [103]:
### OND
guija_ond = data_prep(sa, lon_guija, lat_guija, 'rainfall', [10, 11, 12])
# aggregate
dates, aggr = aggregate(guija_ond)
guija_ond = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(guija_ond).to_netcdf(os.path.join(district_data_path, 'guija_ond.nc'))


### NDJ
guija_ndj = data_prep(sa, lon_guija, lat_guija, 'rainfall', [1, 11, 12]).iloc[1:-2]
# aggregate
dates, aggr = aggregate(guija_ndj)
guija_ndj = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(guija_ndj).to_netcdf(os.path.join(district_data_path, 'guija_ndj.nc'))


### DJF
guija_djf = data_prep(sa, lon_guija, lat_guija, 'rainfall', [1, 2, 12]).iloc[2:-1]
# aggregate
dates, aggr = aggregate(guija_djf)
guija_djf = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(guija_djf).to_netcdf(os.path.join(district_data_path, 'guija_djf.nc'))


### JFM
guija_jfm = data_prep(sa, lon_guija, lat_guija, 'rainfall', [1, 2, 3])
# aggregate
dates, aggr = aggregate(guija_jfm)
guija_jfm = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(guija_jfm).to_netcdf(os.path.join(district_data_path, 'guija_jfm.nc'))

### Marara

In [104]:
### OND
marara_ond = data_prep(sa, lon_marara, lat_marara, 'rainfall', [10, 11, 12])
# aggregate
dates, aggr = aggregate(marara_ond)
marara_ond = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(marara_ond).to_netcdf(os.path.join(district_data_path, 'marara_ond.nc'))


### NDJ
marara_ndj = data_prep(sa, lon_marara, lat_marara, 'rainfall', [1, 11, 12]).iloc[1:-2]
# aggregate
dates, aggr = aggregate(marara_ndj)
marara_ndj = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(marara_ndj).to_netcdf(os.path.join(district_data_path, 'marara_ndj.nc'))


### DJF
marara_djf = data_prep(sa, lon_marara, lat_marara, 'rainfall', [1, 2, 12]).iloc[2:-1]
# aggregate
dates, aggr = aggregate(marara_djf)
marara_djf = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(marara_djf).to_netcdf(os.path.join(district_data_path, 'marara_djf.nc'))


### JFM
marara_jfm = data_prep(sa, lon_marara, lat_marara, 'rainfall', [1, 2, 3])
# aggregate
dates, aggr = aggregate(marara_jfm)
marara_jfm = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(marara_jfm).to_netcdf(os.path.join(district_data_path, 'marara_jfm.nc'))

### Tete

In [105]:
### OND
tete_ond = data_prep(sa, lon_tete, lat_tete, 'rainfall', [10, 11, 12])
# aggregate
dates, aggr = aggregate(tete_ond)
tete_ond = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(tete_ond).to_netcdf(os.path.join(district_data_path, 'tete_ond.nc'))


### NDJ
tete_ndj = data_prep(sa, lon_tete, lat_tete, 'rainfall', [11, 12, 1]).iloc[1:-2]
# aggregate
dates, aggr = aggregate(tete_ndj)
tete_ndj = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(tete_ndj).to_netcdf(os.path.join(district_data_path, 'tete_ndj.nc'))


### DJF
tete_djf = data_prep(sa, lon_tete, lat_tete, 'rainfall', [1, 2, 12]).iloc[2:-1]
# aggregate
dates, aggr = aggregate(tete_djf)
tete_djf = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(tete_djf).to_netcdf(os.path.join(district_data_path, 'tete_djf.nc'))


### JFM
tete_jfm = data_prep(sa, lon_tete, lat_tete, 'rainfall', [1, 2, 3])
# aggregate
dates, aggr = aggregate(tete_jfm)
tete_jfm = pd.DataFrame(aggr, index = dates, columns=['rainfall'])
xarray(tete_jfm).to_netcdf(os.path.join(district_data_path, 'tete_jfm.nc'))